# Class Definitions

The Turing Machine and Tape classes were originally sourced from https://python-course.eu/applications-python/turing-machine.php
although we added support for definitional output and file parsing of transitions among other capabbilities

In [1]:
#
# set constant where to piuck up input files
#
FILE_DIR = '/Users/peter/My Drive/FAU/Computational Theory (COT4420)/turing_machine/config'

In [57]:

class Tape(object):
    
    #blank_symbol = " "
    
    def __init__(self,
                 tape_string = "",
                 blank_symbol = " ", debug=False):
        
        self.__tape = dict((enumerate(tape_string)))
        
        # last line is equivalent to the following three lines:
        #self.__tape = {}
        #for i in range(len(tape_string)):
        #    self.__tape[i] = input[i]
         
        self.__blank_symbol = blank_symbol
        
        if (debug):
            print('Tape::tape[' + str(self.__tape) + ']')
            print('Tape::blank_symbol[' + str(self.__blank_symbol) + ']')
        
        
    def get_blank_symbol(self):
        return self.__blank_symbol
    
    def __str__(self):
        s = ""
        min_used_index = min(self.__tape.keys()) 
        max_used_index = max(self.__tape.keys())
        for i in range(min_used_index, max_used_index+1):
            s += self.__tape[i]
        return s    
   
    def __getitem__(self,index):
        if index in self.__tape:
            return self.__tape[index]
        else:
            return self.__blank_symbol

    def __setitem__(self, pos, char):
        self.__tape[pos] = char 

In [60]:
#
# originally sourced from https://python-course.eu/applications-python/turing-machine.php
# with updates to support serializstion (output) and parsing (input)
#
import string

class TuringMachine(object):
    
  def __init__(self, 
                tape = "", 
                blank_symbol = " ",
                Sigma = " ",
                Gamma = " ",
                states = None,
                initial_state = "",
                final_states = None,
                transition_file = None,
                transition_function = None):
                
    self.__tape = Tape(
       tape_string=tape, 
       blank_symbol=blank_symbol, 
       debug=True)
    
    self.__head_position = 0
    self.__blank_symbol = blank_symbol
    
    self.__Sigma = Sigma
    self.__Gamma = Gamma
    
    self.__states = states
    self.__initial_state =  initial_state
    self.__current_state = initial_state
    
    if (transition_function == None):
        self.read_transition_rules(transition_file)
    else:
        self.__transition_function = transition_function
    
    if final_states == None:
        self.__final_states = set()
    else:
        self.__final_states = set(final_states)


  def get_tape(self): 
    return str(self.__tape)

  def get_tape_blank(self):
     return self.__tape.get_blank_symbol()

  def final(self):
    if self.__current_state in self.__final_states:
        #print('Accepting')
        return True
    else:
        #print('Not Acceping')
        return False
    

  def step(self, debug=False):

    char_under_head = self.__tape[self.__head_position]
    x = (self.__current_state, char_under_head)
    
    if x in self.__transition_function:
        y = self.__transition_function[x]

        if (debug):
          print('transition function: ' + str(x) + ' -> ' + str(y))
          print("tape before operation: " + self.get_tape() + '; [head_pos: ' + str(self.__head_position) + ']')
    
        self.__tape[self.__head_position] = y[1]
        if y[2] == "R":
            self.__head_position += 1
        elif y[2] == "L":
            self.__head_position -= 1
        self.__current_state = y[0]
    else:
       if (debug):
          print('*** WARNING: ' + str(x) + ' not in transition function set ***')
          return False

    if (debug):
      print("tape after operation: " + self.get_tape() + '; [head_pos: ' + str(self.__head_position) + ']')
      print('---------------------------------------------------------')
      
    return True
  
  
  #
  # computes the input_str with a max_ops value as input 
  #
  def compute(self, input_string=None, max_ops=100, debug=False):

    self.__tape = Tape(
       tape_string=input_string, 
       blank_symbol=self.__blank_symbol
       )
    
    if (debug):
       print('TuringMachine::compute(tape=' + self.get_tape() + ', blank_symbol=' + str(self.get_tape_blank()) + ')')
       print('---------------------------------------------------------')

    ops = 0
    while not self.final():
      valid_state = self.step(debug=debug)
      ops += 1

      if not valid_state:
        print('*** not a valid state, therefore not accepting ***')
        return False

      if (ops >= max_ops):
        if (debug):
          print('*** WARNING: not accepting after max_ops('+str(ops)+') operations ***')
        return False
    
    if (debug):
       print("ACCEPTING after " + str(ops) + ' operations.')
       print("TM Tape Output: " + self.get_tape())    

    return True


  #
  # support function for parsing transitions from file
  #
  def split_equation(self, text):
    parts = text.split("=")
    lhs = parts[0].strip()
    rhs = parts[1].strip()
    return lhs, rhs
  

  #
  # support function for parsing transitions from file
  #
  def parse_transition_rule(self, rule):
    
    # Remove whitespace and split into components
    rule = rule.strip().replace(" ", "").replace("δ", "").replace("{", "").replace("}", "")

    lhs, rhs = self.split_equation(rule)    

    lhs = lhs.replace('(', '').replace(')', '')
    rhs = rhs.replace('(', '').replace(')', '')

    # Extract state and symbol from strings
    q, sym = lhs.split(',')
    q_next, sym_next, direction = rhs.split(',')
    
    return (q, sym), (q_next, sym_next, direction)
  
  #
  # read transition rules from file
  # takes filename as input, returns rules
  # 
  def read_transition_rules(self, filename):
    rules = {}
    with open(filename, "r") as f:
        for line in f:
            if line.strip() != "":
                key, value = self.parse_transition_rule(line)
                rules[key] = value
        
        self.__transition_function = rules


  def print(self):
    print("--------------- ** TM Definition ** ---------------")
    print("Q = {" + ", ".join(self.__states) + "}")
    print("Σ = {" + ", ".join(self.__Sigma) + "}")
    print("Γ = {" + ", ".join(self.__Gamma) + "}")
    print("δ : ")
    for key, val in self.__transition_function.items():
        print('   δ'+str(key)+' = {'+str(val)+ str('}'))
    print("q0 = " + self.__initial_state)
    print("b = " + self.__blank_symbol)
    print("F = {" + ", ".join(self.__final_states) + "}")
    print("---------------------------------------------------")


# Examples

## Example 0

In [18]:
states = {"init", "final"}
Sigma = {"1", "0"}
Gamma = {"1", "0", "□"}

initial_state = "init",

accepting_states = ["final"],

transition_function = {("init","0"):("init", "1", "R"),
                       ("init","1"):("init", "0", "R"),
                       ("init","□"):("final","□", "N"),
                       }

final_states = {"final"}

t = TuringMachine(states = states,
                  Gamma = Gamma,
                  Sigma = Sigma,
                  blank_symbol="□",
                  initial_state = "init",
                  final_states = final_states,
                  transition_function=transition_function)

t.print()

if (t.compute("010011001□", debug=True)):
  print("*** Accepting ***")
else:
  print('*** Not Accepting ***')  
    

--------------- ** TM Definition ** ---------------
Q = {init, final}
Σ = {0, 1}
Γ = {0, □, 1}
δ : 
   δ('init', '0') = {('init', '1', 'R')}
   δ('init', '1') = {('init', '0', 'R')}
   δ('init', '□') = {('final', '□', 'N')}
q0 = init
b = □
F = {final}
---------------------------------------------------
TM Tape Input: 010011001
transition function: ('init', '0')
tape before operation: 010011001
tape after operation: 110011001
transition function: ('init', '1')
tape before operation: 110011001
tape after operation: 100011001
transition function: ('init', '0')
tape before operation: 100011001
tape after operation: 101011001
transition function: ('init', '0')
tape before operation: 101011001
tape after operation: 101111001
transition function: ('init', '1')
tape before operation: 101111001
tape after operation: 101101001
transition function: ('init', '1')
tape before operation: 101101001
tape after operation: 101100001
transition function: ('init', '0')
tape before operation: 101100001
tap

## Example 1

In [5]:
#
# From Module 10 slides:
# For the alphabet (Sigma) = {a, b} design a Turing machine 
# that accepts the language denoted by the regular expression 'a*b*'
#
states = {"q0", "q1"}
Sigma = {"1", "0"}
Gamma = {"1", "0", "□"}

initial_state = "q0",
accepting_states = ["q1"], transition_function

final_states = {"q1"}

t = TuringMachine(states=states,
                  Sigma=Sigma,
                  Gamma=Gamma,
                  initial_state="q0",
                  blank_symbol="□",
                  final_states=final_states,
                  transition_function=None,
                  transition_file=FILE_DIR + '/tm10.1')

t.print()

if (t.compute("aaaabbbbbbbb□", debug=True)):
  print("*** Accepting ***")
else:
  print('*** Not Accepting ***')
    

--------------- ** TM Definition ** ---------------
Q = {q1, q0}
Σ = {0, 1}
Γ = {0, □, 1}
δ : 
   δ('q0', 'a') = {('q0', 'b', 'R')}
   δ('q0', 'b') = {('q0', 'b', 'R')}
   δ('q0', '□') = {('q1', '□', 'L')}
q0 = q0
b =  
F = {q1}
---------------------------------------------------
TM Tape Input: aaaabbbbbbbb
transition function: ('q0', 'a')
tape before operation: aaaabbbbbbbb
tape after operation: baaabbbbbbbb
transition function: ('q0', 'a')
tape before operation: baaabbbbbbbb
tape after operation: bbaabbbbbbbb
transition function: ('q0', 'a')
tape before operation: bbaabbbbbbbb
tape after operation: bbbabbbbbbbb
transition function: ('q0', 'a')
tape before operation: bbbabbbbbbbb
tape after operation: bbbbbbbbbbbb
transition function: ('q0', 'b')
tape before operation: bbbbbbbbbbbb
tape after operation: bbbbbbbbbbbb
transition function: ('q0', 'b')
tape before operation: bbbbbbbbbbbb
tape after operation: bbbbbbbbbbbb
transition function: ('q0', 'b')
tape before operation: bbbbbbbbbbb

## Example 2

In [6]:
#
# From Module 10 slides:
# For the alphabet (Sigma) = {0, 1} design a Turing machine 
# that accepts the language denoted by the regular expression '00*'
#
states = {"q0", "q1"}
Sigma = {"1", "0"}
Gamma = {"1", "0", "□"}

initial_state = "q0",
accepting_states = ["q1"], transition_function

final_states = {"q1"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  initial_state = "q0",
                  final_states = final_states,
                  blank_symbol="□",
                  transition_function=None,
                  transition_file=FILE_DIR + '/tm10.2')

t.print()

if (t.compute("000000000000000□", debug=True)):
  print("*** Accepting ***")
else:
  print('*** Not Accepting ***')
    

--------------- ** TM Definition ** ---------------
Q = {q1, q0}
Σ = {0, 1}
Γ = {0, □, 1}
δ : 
   δ('q0', '0') = {('q0', '0', 'R')}
   δ('q0', '□') = {('q1', '□', 'L')}
q0 = q0
b =  
F = {q1}
---------------------------------------------------
TM Tape Input: 000000000000000
transition function: ('q0', '0')
tape before operation: 000000000000000
tape after operation: 000000000000000
transition function: ('q0', '0')
tape before operation: 000000000000000
tape after operation: 000000000000000
transition function: ('q0', '0')
tape before operation: 000000000000000
tape after operation: 000000000000000
transition function: ('q0', '0')
tape before operation: 000000000000000
tape after operation: 000000000000000
transition function: ('q0', '0')
tape before operation: 000000000000000
tape after operation: 000000000000000
transition function: ('q0', '0')
tape before operation: 000000000000000
tape after operation: 000000000000000
transition function: ('q0', '0')
tape before operation: 000000000

## Example 3

In [7]:
#
# From Assignment 7a, L = L(aaba*b)
#
states = {"q0", "q1", "q2", "q3", "q4"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "□", "x", "y", "z", "w"}

initial_state = "q0",
accepting_states = ["q4"]

final_states = {"q4"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  initial_state = "q0",
                  final_states = final_states,
                  blank_symbol="□",
                  transition_function=None,
                  transition_file=FILE_DIR + '/assign7_1a_04122023.txt')

t.print()

if (t.compute("aabaaaaaaaaaaaaaaab□", debug=True)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

--------------- ** TM Definition ** ---------------
Q = {q1, q3, q0, q2, q4}
Σ = {b, a}
Γ = {z, b, w, a, □, x, y}
δ : 
   δ('q0', 'a') = {('q1', 'x', 'R')}
   δ('q1', 'a') = {('q2', 'y', 'R')}
   δ('q2', 'b') = {('q3', 'z', 'R')}
   δ('q3', 'a') = {('q3', 'w', 'R')}
   δ('q3', 'b') = {('q4', '□', 'R')}
q0 = q0
b =  
F = {q4}
---------------------------------------------------
TM Tape Input: aabaaaaaaaaaaaaaaab
transition function: ('q0', 'a')
tape before operation: aabaaaaaaaaaaaaaaab
tape after operation: xabaaaaaaaaaaaaaaab
transition function: ('q1', 'a')
tape before operation: xabaaaaaaaaaaaaaaab
tape after operation: xybaaaaaaaaaaaaaaab
transition function: ('q2', 'b')
tape before operation: xybaaaaaaaaaaaaaaab
tape after operation: xyzaaaaaaaaaaaaaaab
transition function: ('q3', 'a')
tape before operation: xyzaaaaaaaaaaaaaaab
tape after operation: xyzwaaaaaaaaaaaaaab
transition function: ('q3', 'a')
tape before operation: xyzwaaaaaaaaaaaaaab
tape after operation: xyzwwaaaaaaaaaaa

## Example 4

In [10]:
#
# From Assignment 7a, L = L(aaba*b)
#
states = {"q0", "q1", "q2", "q3", "q4", "q5"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "□"}

initial_state = "q0"
accepting_states = ["q5"]

final_states = {"q5"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  blank_symbol="□",
                  initial_state = initial_state,
                  final_states = final_states,
                  transition_file=FILE_DIR + '/assign7_1c_04122023.txt')

t.print()

if (t.compute("ababaaaabbba□", debug=True)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

--------------- ** TM Definition ** ---------------
Q = {q1, q5, q3, q0, q2, q4}
Σ = {b, a}
Γ = {b, □, a}
δ : 
   δ('q0', 'a') = {('q1', 'a', 'R')}
   δ('q0', 'b') = {('q1', 'b', 'R')}
   δ('q1', 'a') = {('q2', 'a', 'R')}
   δ('q1', 'b') = {('q2', 'b', 'R')}
   δ('q2', 'a') = {('q3', 'a', 'R')}
   δ('q2', 'b') = {('q3', 'b', 'R')}
   δ('q3', 'a') = {('q4', 'a', 'R')}
   δ('q3', 'b') = {('q4', 'b', 'R')}
   δ('q4', '□') = {('q5', '□', 'R')}
   δ('q4', 'a') = {('q1', 'a', 'R')}
   δ('q4', 'b') = {('q1', 'b', 'R')}
q0 = q0
b =  
F = {q5}
---------------------------------------------------
TM Tape Input: ababaaaabbba
transition function: ('q0', 'a')
tape before operation: ababaaaabbba
tape after operation: ababaaaabbba
transition function: ('q1', 'b')
tape before operation: ababaaaabbba
tape after operation: ababaaaabbba
transition function: ('q2', 'a')
tape before operation: ababaaaabbba
tape after operation: ababaaaabbba
transition function: ('q3', 'b')
tape before operation: ababaaaabbb

## Example 5

In [61]:
#
# From Assignment 7, 1f L = {anbman+m: n >= 0, m >=1 }
# annswer from textbook pg 429 (Section 9.1 question 8f)
#
states = {"q0", "q1", "q2", "q3", "q4", "q5", "q6"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "y", "Y", "x", "□"}

initial_state = "q0"
accepting_states = ["q6"]

final_states = {"q6"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  blank_symbol="□",
                  initial_state = initial_state,
                  final_states = final_states,
                  transition_file=FILE_DIR + '/assign7_1f_04122023.txt')

t.print()

if (t.compute("aabbbaaaaa□", debug=True)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

Tape::tape[{}]
Tape::blank_symbol[□]
--------------- ** TM Definition ** ---------------
Q = {q1, q5, q3, q0, q2, q4, q6}
Σ = {b, a}
Γ = {b, a, □, x, Y, y}
δ : 
   δ('q0', 'b') = {('q1', 'b', 'L')}
   δ('q0', 'a') = {('q1', 'b', 'R')}
   δ('q1', 'a') = {('q1', 'b', 'R')}
   δ('q1', 'b') = {('q1', 'b', 'L')}
   δ('q1', '□') = {('q2', '□', 'R')}
   δ('q2', 'b') = {('q3', 'Y', 'R')}
   δ('q2', 'x') = {('q5', 'x', 'R')}
   δ('q3', 'x') = {('q3', 'x', 'R')}
   δ('q3', 'b') = {('q3', 'b', 'R')}
   δ('q3', 'a') = {('q4', 'x', 'L')}
   δ('q4', 'b') = {('q4', 'b', 'L')}
   δ('q4', 'x') = {('q4', 'x', 'L')}
   δ('q4', 'y') = {('q2', 'y', 'R')}
   δ('q5', 'x') = {('q5', 'x', 'R')}
   δ('q5', '□') = {('q6', '□', 'L')}
q0 = q0
b = □
F = {q6}
---------------------------------------------------
TuringMachine::compute(tape=aabbbaaaaa□, blank_symbol=□)
---------------------------------------------------------
transition function: ('q0', 'a') -> ('q1', 'b', 'R')
tape before operation: aabbbaaaaa□; [head